In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from nltk.stem.porter import PorterStemmer
import nltk
import re, string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
#from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
# Load the preprocessed data (assuming it's saved in "final_preprocessed_data.csv")
data = pd.read_csv("final_preprocessed_data.csv")

## TF-IDF feature extraction + dimensionality reduction + normalization


In [3]:
# Drop rows where 'Message' column has NaN values
data = data.dropna(subset=['Message'])

# TF-IDF feature extraction
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Set the maximum number of features
X_tfidf = tfidf_vectorizer.fit_transform(data['Message'])

# PCA for dimensionality reduction
pca = PCA(n_components=100)  # Reduce the features to 100 dimensions
X_pca = pca.fit_transform(X_tfidf.toarray())  # Convert the sparse matrix to an array for PCA

# Normalization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_pca)

# Combine the processed features with the labels
final_data = pd.DataFrame(X_scaled)
final_data['label'] = data['label'].values

# Save the final data to a CSV file
final_data.to_csv("final_features_data.csv", index=False)
print("Feature extraction, PCA, and normalization complete. Data saved to final_features_data.csv")


Feature extraction, PCA, and normalization complete. Data saved to final_features_data.csv


In [4]:

# Labels
y = final_data['label']  # Extract labels

# Features (X) - Exclude the 'label' column from the features
X = final_data.drop('label', axis=1)  # Features without the label

# Initialize classifiers
svm = SVC(kernel='linear')  # Support Vector Machine with linear kernel
lr = LogisticRegression(max_iter=1000)  # Logistic Regression
rf = RandomForestClassifier(n_estimators=100, random_state=42)  # Random Forest

# Initialize KFold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

# Function to evaluate a model using KFold
def evaluate_model(model, X, y, kf):
    accuracy_scores = []
    classification_reports = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Use .iloc for DataFrame indexing
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]  # Ensure labels are in the same format

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        accuracy_scores.append(accuracy_score(y_test, predictions))
        classification_reports.append(classification_report(y_test, predictions))  # Print the string report
    
    avg_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    avg_classification_report = classification_reports  # List of classification reports from each fold

    return avg_accuracy, avg_classification_report

# Evaluate using SVM
svm_accuracy, svm_report = evaluate_model(svm, X, y, kf)
print("SVM Average Accuracy:", svm_accuracy)
print("SVM Classification Report:\n", "\n".join(svm_report))  # Print each fold's report

# Evaluate using Logistic Regression
lr_accuracy, lr_report = evaluate_model(lr, X, y, kf)
print("Logistic Regression Average Accuracy:", lr_accuracy)
print("Logistic Regression Classification Report:\n", "\n".join(lr_report))  # Print each fold's report

# Evaluate using Random Forest
rf_accuracy, rf_report = evaluate_model(rf, X, y, kf)
print("Random Forest Average Accuracy:", rf_accuracy)
print("Random Forest Classification Report:\n", "\n".join(rf_report))  # Print each fold's report


SVM Average Accuracy: 0.8009017622127264
SVM Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.91      0.83     13561
           1       0.88      0.68      0.77     12211

    accuracy                           0.80     25772
   macro avg       0.82      0.80      0.80     25772
weighted avg       0.82      0.80      0.80     25772

              precision    recall  f1-score   support

           0       0.77      0.90      0.83     13845
           1       0.86      0.68      0.76     11926

    accuracy                           0.80     25771
   macro avg       0.81      0.79      0.79     25771
weighted avg       0.81      0.80      0.80     25771

              precision    recall  f1-score   support

           0       0.76      0.90      0.83     13592
           1       0.86      0.69      0.77     12179

    accuracy                           0.80     25771
   macro avg       0.81      0.80      0.80     25771
weigh